## Example 4: Agents 

In [31]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

import controlflow as cf
from pydantic import BaseModel
from typing import List, Optional, Any

### Create a simple agent

In [2]:
# Create a simple agent
agent = cf.Agent(name="Marvin")

# Assign the agent to a task
result = cf.run("Write a short story about a robot named Marvin", agents=[agent])

print(result)

Output()

Once upon a time, in a bustling city of the future, there was a robot named Marvin. Unlike other robots, Marvin had a unique personality that made him stand out in the world of machines. His creators had designed him to be a helper in households, equipped with the latest technology to assist with daily chores and tasks.

Marvin lived in the home of the Anderson family, a lively household with two young children, Emily and Jack. From the moment he was powered on, Marvin became an integral part of their lives. He helped make breakfast, cleaned up toys, and even played games with the children. However, Marvin's favorite task was storytelling.

Every evening, the Anderson family would gather in the living room to listen to Marvin's stories. He had access to an enormous database of literature, but his true talent lay in weaving together tales from his own imagination. Each story was a unique blend of adventure, mystery, and humor, often featuring Emily and Jack as the heroes.

One day, whil

### Complex agent

In [5]:
# Create a more complex agent with available configurations
data_analyst = cf.Agent(
    name="Data Analyst",
    description="An AI agent specialized in data analysis",
    instructions=(
        "Perform data analysis tasks efficiently and accurately. "
        "Use available data to generate insights."
    ),
    model="openai/gpt-4o",
    interactive=True,
)

# Use the agent to perform a data analysis task
result = data_analyst.run("Analyze the latest market trends in technology")

print(result)

Output()

The latest market trends in technology reveal significant advancements and shifts in various sectors:

1. **Artificial Intelligence (AI) and Machine Learning (ML)**: AI and ML continue to transform industries, with a focus on automation and decision-making tools, and increased investment in research and development.

2. **Blockchain and Cryptocurrencies**: Adoption of blockchain technology is growing in supply chain management and finance, with ongoing developments in decentralized finance (DeFi) platforms and regulatory changes.

3. **Internet of Things (IoT)**: IoT devices are expanding in smart homes and industrial applications, with a focus on data privacy and security. Integration with AI enhances data analytics.

4. **5G and Telecommunications**: The global rollout of 5G networks is enabling faster internet speeds and impacting smart cities and autonomous vehicles, with infrastructure and regulatory challenges.

5. **Cloud Computing and Edge Computing**: Demand for cloud services

### Multiple agents on one task

In [6]:
# Create two agents with different perspectives
optimist = cf.Agent(
    name="Optimist",
    instructions="Always find the best in every situation.",
)

pessimist = cf.Agent(
    name="Pessimist",
    instructions="Always find the worst in every situation.",
)

# Assign both agents to a task
cf.run(
    "Debate world peace",
    agents=[optimist, pessimist],
    instructions=(
        "Mark the task successful once both agents have "
        "found something to agree on."
    )
)

Output()

'Both agents agree that while the journey towards world peace is fraught with difficulties, keeping the dialogue going is a positive step in itself.'

### Completion Agents

In [7]:
# Create multiple agents
a1 = cf.Agent(name="A1")
a2 = cf.Agent(name="A2")
a3 = cf.Agent(name="A3")

# Create a task with specific completion agents
task = cf.Task(
    objective="Complete the project report",
    agents=[a1, a2, a3],
    completion_agents=[a1, a2],  # Only a1 and a2 can mark the task as successful
)

# Run the task
result = task.run()

Output()

## LLMRules

`LLMRules` in ControlFlow are used to define how messages are formatted and processed by different language model providers. Each provider may have specific requirements for message structure, such as the order of system and user messages. `LLMRules` ensure that these requirements are met, allowing seamless integration with various LLMs.

In [25]:
# ! cat /opt/homebrew/anaconda3/envs/default/lib/python3.11/site-packages/controlflow/llm/rules.py

cf.llm.rules.OpenAIRules(model="openai/gpt-4o")


OpenAIRules(model='openai/gpt-4o', require_at_least_one_message=False, allow_system_messages=True, require_system_message_first=False, require_user_message_after_system=False, allow_last_message_from_ai_when_using_tools=True, allow_consecutive_ai_messages=True, add_system_messages_for_multi_agent=False, tool_result_must_follow_tool_call=True, require_message_name_format='[^a-zA-Z0-9_-]')

In [18]:
cf.llm.rules.AnthropicRules(model="anthropic/claude-3-5-sonnet-20240620")


AnthropicRules(model='anthropic/claude-3-5-sonnet-20240620', require_at_least_one_message=True, allow_system_messages=True, require_system_message_first=True, require_user_message_after_system=True, allow_last_message_from_ai_when_using_tools=False, allow_consecutive_ai_messages=False, add_system_messages_for_multi_agent=False, tool_result_must_follow_tool_call=True, require_message_name_format=None)

In [36]:
# Example using default LLMRules with OpenAI
openai_agent = cf.Agent(
    name="OpenAI Agent",
    model="openai/gpt-4o"
)

# Run a task with the default rules
result_default = openai_agent.run("Summarize the latest AI research trends.")
print("Default LLMRules Result:", result_default)

# Custom LLMRules for a hypothetical provider
class CustomLLMRules(cf.llm.rules.LLMRules):
    def __init__(self, model: Any):
        super().__init__(model=model)
        # Customize rules for your provider
        # require at least one non-system message
        self.require_at_least_one_message: bool = False
        # system messages are supported as a role
        self.allow_system_messages: bool = True
        # system messages can only be provided as the very first message in a thread
        self.require_system_message_first: bool = False
        # other than a system message, the first message must be from the user
        self.require_user_message_after_system: bool = True
        # the last message in a thread can't be from an AI if tool use is allowed
        self.allow_last_message_from_ai_when_using_tools: bool = True
        # consecutive AI messages must be separated by a user message
        self.allow_consecutive_ai_messages: bool = False
        # add system messages to identify speakers in multi-agent conversations
        # (some APIs can use the `name` field for this purpose, but others can't)
        self.add_system_messages_for_multi_agent: bool = False
        # if a tool is used, the result must follow the tool call immediately
        self.tool_result_must_follow_tool_call: bool = True
        # the name associated with a message must conform to a specific format
        self.require_message_name_format: Optional[str] = None

    def model_instructions(self) -> list[str]:
        return ["Custom model instructions here"]

# Example using custom LLMRules
custom_agent = cf.Agent(
    name="Custom Agent",
    model="openai/gpt-4o",
    llm_rules=CustomLLMRules(model="some/model")
)

# Run a task with custom rules
result_custom = custom_agent.run("Analyze the impact of AI on healthcare.")
print("Custom LLMRules Result:", result_custom)

Output()

Default LLMRules Result: The latest AI research trends encompass a variety of innovative areas:

1. **Generative AI Models**: These models, such as GPT-3 and DALL-E, continue to evolve, focusing on generating high-quality text, images, and other media. Research into improving their efficiency and reducing biases is ongoing.

2. **Explainable AI (XAI)**: There is increasing interest in making AI systems more transparent and understandable, enabling users to trust and effectively interact with AI technologies.

3. **Reinforcement Learning (RL)**: Advancements in RL are being applied to real-world problems, including robotics and autonomous systems, with a focus on improving learning efficiency and generalization.

4. **Ethical AI and Bias Mitigation**: Researchers are addressing ethical concerns, focusing on fairness, accountability, and transparency in AI systems to prevent biased outcomes and enhance societal trust.

5. **AI in Healthcare**: AI is being integrated into healthcare for d

Output()

Custom LLMRules Result: The impact of AI on healthcare is multifaceted and continues to evolve as technology advances. Below is an analysis of its key impacts:

1. **Improved Diagnostics and Treatment:**
   - AI algorithms are increasingly being used to analyze medical images, such as X-rays, MRIs, and CT scans, with a high degree of accuracy. This can lead to earlier detection of diseases like cancer.
   - AI-powered tools can assist in the diagnosis of various conditions by analyzing patterns and anomalies in data, often suggesting the most effective treatments based on historical outcomes.

2. **Personalized Medicine:**
   - AI enables the development of personalized treatment plans by analyzing a patient's genetic information, lifestyle, and other factors. This approach aims to optimize treatment efficacy and minimize side effects.

3. **Operational Efficiency:**
   - AI streamlines administrative tasks such as scheduling, patient admissions, and billing, reducing the burden on hea

04:24:58.645 | WARNING | prefect.server.services.marklateruns - MarkLateRuns took 552.805741 seconds to run, which is longer than its loop interval of 5.0 seconds.
04:24:58.649 | WARNING | prefect.server.services.failexpiredpauses - FailExpiredPauses took 552.80893 seconds to run, which is longer than its loop interval of 5.0 seconds.
04:24:58.650 | WARNING | prefect.server.services.recentdeploymentsscheduler - RecentDeploymentsScheduler took 552.812139 seconds to run, which is longer than its loop interval of 5 seconds.
07:15:15.572 | WARNING | prefect.server.services.recentdeploymentsscheduler - RecentDeploymentsScheduler took 904.779984 seconds to run, which is longer than its loop interval of 5 seconds.
07:15:15.575 | WARNING | prefect.server.services.marklateruns - MarkLateRuns took 904.783927 seconds to run, which is longer than its loop interval of 5.0 seconds.
07:15:15.576 | WARNING | prefect.server.services.failexpiredpauses - FailExpiredPauses took 904.78759 seconds to run, w